In [23]:
# Importing modules
import cv2
from numpy.fft import fft2, fftshift, ifft2, ifftshift
from numpy.fft import fft, ifft
from numpy import tile, real, min, zeros
import numpy as np
import pandas as pd


from scipy.signal import firwin
from skimage.filters import gaussian

import math
import time

M_PI = math.pi
eps = 2 ** (-52)

raw_data = []

# 확대처리 대상 설정
# videoName = '200Hz_sr2000_0113' #대상 비디오 파일명
# videoName = 'baby_roi' #대상 비디오 파일명
# videoName = '2Hz'
# videoType = '.mp4'  # 확장자
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # 확대과정 후 저장시 mp4로 저장

# Parameter Initializing
Low_Freq = 1.5
High_Freq = 2.5
alpha = 5
Sampling_Rate = 30
refFrame = 0
NumberOfPyramid = 2
Orientation = 2
sigma = 0
attenuateOtherFreq = 0  # 영역 외 주파수에 대해 베재할 것인가에 대해 True or False
ratioOfFrame = 1

# Video File Importing
# fileName = './input/{}{}'.format(videoName, videoType)
# cap_original = cv2.VideoCapture(fileName)
cap_original = cv2.VideoCapture(0)

# Video Information Checking
ret, frame = cap_original.read()

frame = cv2.resize(frame, dsize = (240, 160), interpolation = cv2.INTER_AREA)
# frame = cv2.pyrDown(frame) # (320, 240)

# frame = cv2.pyrDown(frame) # (160,120)

frameX = frame.shape[1]
frameY = frame.shape[0]
nColor = frame.shape[2]
# nFrame = int(cap_original.get(cv2.CAP_PROP_FRAME_COUNT))

pyrLayers = NumberOfPyramid
rVals = [1]
for i in range(pyrLayers):
    rVals.append(0.5 ** (i + 1))


def bgr2yiq(img):
    R = img[:, :, 2]
    G = img[:, :, 1]
    B = img[:, :, 0]

    Y = 0.299 * R + 0.587 * G + 0.114 * B
    I = 0.596 * R - 0.274 * G - 0.322 * B
    Q = 0.211 * R - 0.523 * G + 0.312 * B

    img_yiq = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_yiq[:, :, 0] = Y
    img_yiq[:, :, 1] = I
    img_yiq[:, :, 2] = Q
    # YIQ = np.array([[0.299, 0.587, 0.114],[0.596, -0.274, -0.322],[0.211, -0.523, 0.312]])
    return img_yiq


def yiq2bgr(img):
    Y = img[:, :, 0]
    I = img[:, :, 1]
    Q = img[:, :, 2]

    R = Y + 0.956 * I + 0.621 * Q
    G = Y - 0.272 * I - 0.647 * Q
    B = Y - 1.106 * I + 1.703 * Q

    img_rgb = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_rgb[:, :, 0] = B
    img_rgb[:, :, 1] = G
    img_rgb[:, :, 2] = R
    # YIQ = np.array([[0.299, 0.587, 0.114],[0.596, -0.274, -0.322],[0.211, -0.523, 0.312]])
    return img_rgb


def getFilters(dimension, rVals, orientations):
    X = int(dimension[1])
    Y = int(dimension[0])

    defaultTwidth = 1
    twidth = defaultTwidth

    polarGrid = getPolarGrid(dimension)
    count = 0

    angle = np.array(polarGrid[0])  # 픽셀 위치별 각도 값
    rad = np.array(polarGrid[1])  # 픽셀 위치별 거리 값

    mask = getRadialMaskPair(rVals[0], rad, twidth)
    himask = mask[0]
    lomaskPrev = mask[1]

    filters = []
    filters.append(himask)

    for k in range(1, len(rVals)):
        mask = getRadialMaskPair(rVals[k], rad, twidth)
        himask = mask[0]
        lomask = mask[1]

        radMask = himask * lomaskPrev

        for j in range(1, orientations + 1):
            angleMask = getAngleMask(j, orientations, angle)
            filters.append(radMask * angleMask / 2)

        lomaskPrev = lomask
    filters.append(lomask)

    for k in range(len(filters)):
        filters[k] = np.array(filters[k])
    return filters


# 화면의 너비x높이 크기의 극좌표르를 2차원 배열으로 형성한다.
def getPolarGrid(dimension):
    X = dimension[1]
    Y = dimension[0]
    centerX = int(X / 2)
    centerY = int(Y / 2)

    # Create rectangular grid
    xramp = np.array([[(x - int(X / 2)) / (X / 2) for x in range(X)] for y in range(Y)])
    yramp = np.array([[(y - int(Y / 2)) / (Y / 2) for x in range(X)] for y in range(Y)])
    angle = np.arctan2(xramp, yramp) + M_PI / 2

    rad = np.sqrt(xramp ** 2 + yramp ** 2)
    rad[centerY][centerX] = rad[centerY - 1][centerX]

    polarGrid = [angle, rad]
    return polarGrid


# 3차원 그래프상으로 보이는 완전 꼬깔 형태에서, 위 아래의 범위를 잘라내고 원뿔대 같은 형태도 만든다.
def getRadialMaskPair(r, rad, twidth):
    X = int(rad.shape[1])
    Y = int(rad.shape[0])

    log_rad = np.log2(rad) - np.log2(r)

    himask = log_rad
    himask[himask > 0] = 0
    himask[himask < -twidth] = -twidth
    himask = himask * M_PI / (2 * twidth)

    himask = np.cos(himask)
    lomask = np.sqrt(1 - himask ** 2)

    mask = [himask, lomask]
    return mask


def getAngleMask(b, orientations, angle):
    order = orientations - 1
    const = (2 ** (2 * order)) * (math.factorial(order) ** 2) / (
                orientations * math.factorial(2 * order))  # Scaling constant

    angle_ = (M_PI + angle - (M_PI * (b - 1) / orientations)) % (2 * M_PI) - M_PI
    anglemask = 2 * np.sqrt(const) * (np.cos(angle_) ** order) * (abs(angle_) < (M_PI / 2))  # Make falloff smooth
    return anglemask


def getFilterIDX2(filters, orientations, rVals):
    X = filters[0].shape[1]
    Y = filters[0].shape[0]
    nFilts = len(filters)
    filtIDX = [[None for j in range(orientations)] for i in range(nFilts)]
    croppedFilters = []

    # himask IDX
    filtIDX[0][0] = [y for y in range(Y)]
    filtIDX[0][1] = [x for x in range(X)]
    croppedFilters.append(filters[0])

    # stearable filter IDX
    for k in range(1, nFilts - 1, orientations):
        n = int(k / 2) + 1
        lb_y = int((Y * (sum(rVals[0:n]) - 1)) / 2)
        ub_y = Y - lb_y
        lb_x = int((X * (sum(rVals[0:n]) - 1)) / 2)
        ub_x = X - lb_x

        for i in range(orientations):
            filtIDX[k + i][0] = [y + lb_y for y in range(ub_y - lb_y)]
            filtIDX[k + i][1] = [x + lb_x for x in range(ub_x - lb_x)]

        for i in range(orientations):
            croppedFilters.append(filters[k + i][lb_y:ub_y, lb_x:ub_x])

    # lomaskIDX
    lb_y = int((Y * (sum(rVals) - 1)) / 2)
    ub_y = Y - lb_y
    lb_x = int((X * (sum(rVals) - 1)) / 2)
    ub_x = X - lb_x

    filtIDX[nFilts - 1][0] = [y + lb_y for y in range(ub_y - lb_y)]
    filtIDX[nFilts - 1][1] = [x + lb_x for x in range(ub_x - lb_x)]
    croppedFilters.append(filters[nFilts - 1][lb_y:ub_y, lb_x:ub_x])

    filterIDX = [croppedFilters, filtIDX]
    return filterIDX


def fir_window_bp_2(delta, fl, fh):
    length = delta.shape[0] + 1
    b = firwin(length, (fl * 2, fh * 2), pass_zero=False)[0:length - 1]

    m = delta.shape[1]
    batches = 20
    batch_size = int(m / batches) + 1
    temp = fft(ifftshift(b))
    out = zeros(delta.shape, dtype=delta.dtype)

    i = 0
    indexes = (batch_size * i, min((batch_size * (i + 1), m)))
    while (indexes[0] < m):
        indexes = (batch_size * i, min((batch_size * (i + 1), m)))
        freq = fft(delta[:, indexes[0]:indexes[1]], axis=0) * tile(temp, (
            delta.shape[2], indexes[1] - indexes[0], 1)).swapaxes(0, 2)
        out[:, indexes[0]:indexes[1]] = real(ifft(freq, axis=0))
        i += 1
        indexes = (batch_size * i, min((batch_size * (i + 1), m)))
    return out


def iir_window_bp_2(delta, fl, fh):
    length = delta.shape[0] + 1
    b = firwin(length, (fl * 2, fh * 2), pass_zero=False)[0:length - 1]

    out = np.apply_along_axis(lambda m: np.convolve(m, ifftshift(b), mode='same'), axis=0, arr=delta)
    return out


def amplitude_weighted_blur(x, weight, sigma):
    if sigma != 0:
        return gaussian(x * weight, sigma, mode="wrap") / gaussian(weight, sigma, mode="wrap")
    return x


#################################################################
#################################################################

filters = getFilters([frameY, frameX], rVals, Orientation)

filterIDX = getFilterIDX2(filters, Orientation, rVals)
croppedFilters = filterIDX[0]
filtIDX = filterIDX[1]

########
loopRange = 5
vidFFT = np.zeros([loopRange, frameY, frameX], dtype=np.complex64)
magnifiedLumaFFT = np.zeros([loopRange, frameY, frameX], dtype=np.complex64)
numLevels = len(filters)
resultSection = []
frameYIQ = []

bpFilter = firwin(loopRange, (2 * Low_Freq / Sampling_Rate, 2 * High_Freq / Sampling_Rate), pass_zero=False)

# initialization
vidFFT = np.zeros([loopRange, frameY, frameX], dtype=np.complex64)
magnifiedLumaFFT = np.zeros([loopRange, frameY, frameX], dtype=np.complex64)
for k in range(0, loopRange):
    cv2.imshow('monitor', frame)
    clipMat = frame / 255
    frameYIQ.append(bgr2yiq(clipMat))
    vidFFT[k] = fftshift(fft2(frameYIQ[k][:, :, 0]))
    ret, frame = cap_original.read()
    frame = cv2.resize(frame, dsize = (240, 160), interpolation = cv2.INTER_AREA)
#     frame = cv2.pyrDown(frame)
#     frame = cv2.pyrDown(frame)
    
cv2.destroyAllWindows()


 
print(frame.shape)
# cam monitoring
while (1):
    tic = time.time()
    if ret == False:
        break

    vidFFTPast = vidFFT
    vidFFT = np.zeros([loopRange, frameY, frameX], dtype=np.complex64)
    magnifiedLumaFFT = np.zeros([loopRange, frameY, frameX], dtype=np.complex64)
    magnifiedLumaFFTNow = np.zeros([frameY, frameX], dtype=np.complex64)

    clipMat = frame / 255
    # renew frameYIQ
    frameYIQ = frameYIQ[1:loopRange]
    frameYIQ.append(bgr2yiq(clipMat))

    # renew vidFFT
    vidFFT[0:vidFFT.shape[0] - 1] = vidFFTPast[1:vidFFT.shape[0]]
    vidFFT[vidFFT.shape[0] - 1] = fftshift(fft2(frameYIQ[loopRange - 1][:, :, 0]))

    for level in range(1, numLevels - 1):
        X = len(croppedFilters[level][0])
        Y = len(croppedFilters[level])
        lb_x = filtIDX[level][1][0]
        ub_x = filtIDX[level][1][-1] + 1
        lb_y = filtIDX[level][0][0]
        ub_y = filtIDX[level][0][-1] + 1

        # 1. 기준프레임 설정
        clipMat = croppedFilters[level] * vidFFT[refFrame][lb_y:ub_y, lb_x:ub_x]
        pyrRef = ifft2(ifftshift(clipMat))
        pyrRefPhaseOrig = pyrRef / abs(pyrRef)
        pyrRef = np.angle(pyrRef)

        delta = np.zeros([loopRange, Y, X], dtype=np.float16)

        # 2. 각 프레임간 차이 계산
        for frameIDX in range(0, loopRange):
            filterResponse = ifft2(ifftshift(croppedFilters[level] * vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]))
            pyrCurrent = np.angle(filterResponse)
            clipMat1 = pyrCurrent - pyrRef
            clipMat2 = M_PI + clipMat1
            clipMat3 = clipMat2 % (2 * M_PI)
            clipMat4 = clipMat3 - M_PI
            clipMat = clipMat4
            delta[frameIDX] = clipMat

        # 3. 픽셀 변화 양상에 대한 band pass filtering
        # delta_1 = np.apply_along_axis(lambda m: np.convolve(m, ifftshift(bpFilter), mode='same'), axis=0, arr=delta)
        delta_filtered = np.sum(delta*bpFilter.reshape(loopRange,1,1),axis=0)

        # for frameIDX in range(0, loopRange):
        #     Phase = delta_1[frameIDX]
        #
        #     originalLevel = ifft2(ifftshift(croppedFilters[level] * vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]))
        #
        #     if (sigma != 0):
        #         Phase = amplitude_weighted_blur(delta_1[frameIDX], abs(originalLevel) + eps, sigma)
        #
        #     # 4. alpha 계수 곱 및 병합
        #     Phase = alpha * Phase
        #
        #     if (attenuateOtherFreq):
        #         tempOrig = abs(originalLevel) * pyrRefPhaseOrig
        #     else:
        #         tempOrig = originalLevel
        #
        #     tempTransformOut = np.exp(1j * Phase) * tempOrig
        #
        #     A = croppedFilters[level]
        #     B = fftshift(fft2(tempTransformOut))
        #     curLevelFrame = 2 * A * B
        #
        #     matClip = magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]
        #     magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] = matClip + curLevelFrame

        Phase = delta_filtered
        originalLevel = ifft2(ifftshift(croppedFilters[level] *
                                        vidFFT[loopRange//2][lb_y:ub_y, lb_x:ub_x]))

        if (sigma != 0):
            Phase = amplitude_weighted_blur(delta_filtered, abs(originalLevel) + eps, sigma)

        # 4. alpha 계수 곱 및 병합
        Phase = alpha * Phase

        if (attenuateOtherFreq):
            tempOrig = abs(originalLevel) * pyrRefPhaseOrig
        else:
            tempOrig = originalLevel

        tempTransformOut = np.exp(1j * Phase) * tempOrig

        A = croppedFilters[level]
        B = fftshift(fft2(tempTransformOut))
        curLevelFrame = 2 * A * B
        matClip = magnifiedLumaFFTNow[lb_y:ub_y, lb_x:ub_x]
        magnifiedLumaFFTNow[lb_y:ub_y, lb_x:ub_x] = matClip + curLevelFrame

    # 5. lowpass residual 처리
    level = len(filters) - 1
    lb_x = filtIDX[level][1][0]
    ub_x = filtIDX[level][1][-1] + 1
    lb_y = filtIDX[level][0][0]
    ub_y = filtIDX[level][0][-1] + 1

    lowpassFrame = vidFFT[loopRange//2][lb_y:ub_y, lb_x:ub_x] * (croppedFilters[level] ** 2)
    magnifiedLumaFFTNow[lb_y:ub_y, lb_x:ub_x] += lowpassFrame

    magnifiedLuma = np.real(ifft2(ifftshift(magnifiedLumaFFTNow)))
    # 명암에 대한 데이터만 적용하기 위해 yiq변환해서 Y 값만 가져온다.
    frameYIQ[loopRange//2][:, :, 0] = magnifiedLuma
    outFrame = yiq2bgr(frameYIQ[loopRange//2]) * 255

    outFrame[outFrame > 255] = 255
    outFrame[outFrame < 0] = 0
    outFrame = np.uint8(outFrame)
    
    outFrame = cv2.flip(outFrame, -1)
    frame = cv2.flip(frame, -1)
    
    cv2.imshow('pbmMonitor', outFrame)
    cv2.imshow('original', frame)
    
    ret, frame = cap_original.read()
    frame = cv2.resize(frame, dsize = (240, 160), interpolation = cv2.INTER_AREA)
#     frame = cv2.pyrDown(frame)
#     frame = cv2.pyrDown(frame)
    
    take_time = time.time() - tic
    take_time = round(take_time, 4)
    
    print('iter done / {:.3f} sec'.format(time.time() - tic))
    raw_data.append(take_time)
    
    
    if cv2.waitKey(1) & 0xFF == 27: 
        break

#########################################
print(raw_data)
xlxs_dir='sample.xlsx'
raw_data1 = pd.DataFrame(raw_data)
with pd.ExcelWriter(xlxs_dir) as writer:
    raw_data1.to_excel(writer, sheet_name = 'raw_data1') #raw_data1 시트에 저장

cv2.destroyAllWindows()
cap_original.release()

(160, 240, 3)
iter done / 0.217 sec
iter done / 0.085 sec
iter done / 0.087 sec
iter done / 0.071 sec
iter done / 0.079 sec
iter done / 0.097 sec
iter done / 0.080 sec
iter done / 0.081 sec
iter done / 0.096 sec
iter done / 0.081 sec
iter done / 0.075 sec
iter done / 0.099 sec
iter done / 0.093 sec
iter done / 0.078 sec
iter done / 0.095 sec
iter done / 0.082 sec
iter done / 0.096 sec
iter done / 0.073 sec
iter done / 0.097 sec
iter done / 0.088 sec
iter done / 0.085 sec
iter done / 0.099 sec
iter done / 0.109 sec
iter done / 0.087 sec
iter done / 0.094 sec
iter done / 0.110 sec
iter done / 0.116 sec
iter done / 0.093 sec
iter done / 0.101 sec
iter done / 0.110 sec
iter done / 0.101 sec
iter done / 0.106 sec
iter done / 0.088 sec
iter done / 0.075 sec
iter done / 0.087 sec
iter done / 0.083 sec
iter done / 0.096 sec
iter done / 0.103 sec
iter done / 0.101 sec
iter done / 0.099 sec
iter done / 0.104 sec
iter done / 0.104 sec
iter done / 0.116 sec
iter done / 0.096 sec
iter done / 0.085 